<p align="left">
  <img src="./logo_UTN.svg" />
</p>


# **Teoría de Circuitos II - R4001 - 2023**
## Trabajo Semanal 4
### Autor: Bruno Glecer

# Consigna

La tarea semanal consiste en diseñar un filtro pasa-banda con las siguientes caracteristicas

- Frecuencia de corte inferior $f_{ci} = 1600\mathrm{kHz}$
- Frecuencia de corte superior $f_{cs} = 2500\mathrm{kHz}$
- Ripple maximo en la banda de paso: $3\mathrm{dB}$
- Maxima planicidad en la banda de paso
- Ganancia maxima en la banda de paso: $10\mathrm{dB}$
- Atenuacion minima de $\alpha_{min} = 20\mathrm{dB}$ a las frecuencias $1250\mathrm{kHz}$ y $3200\mathrm{kHz}$
Se piden las siguientes consignas:

Se pide:

1. Obtener la funcion de transferencia normalizada del filtro
2. Graficar el diagrama de polos y ceros
3. Graficar la transferencia (modulo y fase) del filtro pedido
4. Sintetizar el filtro utilizando estructuras Ackeberg-Mossberg
5. Simular el filtro obtenido, verificando las especificaciones de diseño.



# 1. Transferencia

Comenzamos normalizando los parametros del filtro usando la frecuencia central $\omega_0 = \sqrt{\omega_{ci} \omega_{cs}}$ como norma para diseñar el pasabajos prototipo y luego aplicar la transformacion en frecuencia utilizando el nucleo $p = K(s) = Q\dfrac{s^2 + 1}{s}$

In [1]:
import numpy as np

#Uso los nombres de f_1 y f_2 para las frecuencias de paso y f_s1 y f_s2 para las frecuencias de corte

f_p1 = 1600*10**3
f_p2 = 2500*10**3

f_s1 = 1250*10**3
f_s2 = 3200*10**3

#Caluclos de los parametros del filtro

f_0 = np.sqrt(f_p1*f_p2)
bw = f_p2 - f_p1
q_filtro = f_0/bw

#Paso a frecuencia angular y normalizo

w_norma = f_0 * 2*np.pi

w_0  = 1

w_p1 = f_p1 * 2*np.pi / w_norma
w_p2 = f_p2 * 2*np.pi / w_norma

w_s1 = f_s1 * 2*np.pi / w_norma
w_s2 = f_s2 * 2*np.pi / w_norma

print(f"Frecuencias de paso: ({w_p1} ,  {w_p2})")
print(f"Frecuencias de corte: ({w_s1}  a  {w_s2})")

Frecuencias de paso: (0.7999999999999999 ,  1.2500000000000002)
Frecuencias de corte: (0.6250000000000001  a  1.5999999999999999)


Para obtener los parametros del filtro prototipo que debemos diseñar, tenemos que elegir los valores de frecuencia tal que imponga las restricciones mas fuertes en el caso de que la plantilla no sea simetrica.

In [2]:
#Transformo los parametros del filtro pasabanda al filtro prototipos usando el nucleo de transformacion:

def nucleo_wi(w):
    return q_filtro*((w**2 - 1)/w) #El nucleo es diferente al definido porque esta formula es para transformar el valor absoluto de frecuencia imaginaria, no compleja.

w_p1_lp = nucleo_wi(w_p1)
w_p2_lp = nucleo_wi(w_p2)

w_s1_lp = nucleo_wi(w_s1)
w_s2_lp = nucleo_wi(w_s2)


print(f"Frecuencias de paso: ({w_p1_lp} ,  {w_p2_lp})")
print(f"Frecuencias de corte: ({w_s1_lp}  a  {w_s2_lp})")



Frecuencias de paso: (-1.0000000000000004 ,  1.000000000000001)
Frecuencias de corte: (-2.1666666666666656  a  2.1666666666666665)


Seguimos eligiendo los valores de frecuencias para el pasa bajos prototipo de forma que ponga la mayor exigencia posible

In [3]:
w_p_lp = min(abs(w_p1_lp), abs(w_p2_lp))
w_s_lp = min(abs(w_s1_lp), abs(w_s2_lp))

print(f"Frecuencia de paso: {w_p_lp}")
print(f"Frecuencia de corte: {w_s_lp}")



Frecuencia de paso: 1.0000000000000004
Frecuencia de corte: 2.1666666666666656
